In [4]:
import pandas as pd
import re
import sys
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
#from bs4 import BeautifulSoup
import re
import pandas as pd
#from tabulate import tabulate
import os
#import chromedriver_binary
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import datetime as dt
from dateutil.parser import parse

In [5]:
AS= pd.read_csv('../data/Atlantic_Southeast.csv',skiprows=7)


In [6]:
df= pd.DataFrame(AS)
print(len(df))

2


In [7]:
df.loc[df['Actual Arrival Time']=='00:00']

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
0,EV,10/27/2017,5363.0,N738EV,CVG,01:21,00:00


In [8]:
df= df.dropna(subset=['Scheduled Arrival Time','Actual Arrival Time'])
df.head()

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
0,EV,10/27/2017,5363.0,N738EV,CVG,01:21,00:00


In [9]:
#Rename it to scheduled arrival date
df.rename(columns = {'Date (MM/DD/YYYY)':'Scheduled Date'}, inplace = True) 


In [10]:
df.head()

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
0,EV,10/27/2017,5363.0,N738EV,CVG,01:21,00:00


In [11]:
df['Scheduled Date']= pd.to_datetime(df['Scheduled Date'])
type(df['Scheduled Date'][0])
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time
0,EV,2017-10-27,5363.0,N738EV,CVG,01:21,00:00


In [12]:
# Makea new column called actual arrival date 
df['Actual Arrival Date'] = df['Scheduled Date']
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date
0,EV,2017-10-27,5363.0,N738EV,CVG,01:21,00:00,2017-10-27


In [13]:
df.loc[df['Actual Arrival Time']=='24:00']

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date


In [14]:
df['Actual Arrival Time']= df['Actual Arrival Time'].str.replace('24:00','00:00')

In [15]:
df.loc[df['Actual Arrival Time']=='00:00']

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date
0,EV,2017-10-27,5363.0,N738EV,CVG,01:21,00:00,2017-10-27


In [16]:
df.loc[df['Actual Arrival Time']=='00:00','Actual Arrival Date']+=dt.timedelta(days=1)

In [17]:
df['Actual Arrival Date']= df['Actual Arrival Date'].dt.strftime('%Y-%m-%d')

In [18]:
df['Scheduled Date']= df['Scheduled Date'].dt.strftime('%Y-%m-%d')

In [19]:
df['Actual Arrival']=df['Actual Arrival Date'] + ' ' + df['Actual Arrival Time']

In [20]:
df['Scheduled Arrival']=df['Scheduled Date'] + ' ' + df['Scheduled Arrival Time']
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date,Actual Arrival,Scheduled Arrival
0,EV,2017-10-27,5363.0,N738EV,CVG,01:21,00:00,2017-10-28,2017-10-28 00:00,2017-10-27 01:21


In [21]:
FMT = '%Y-%m-%d %H:%M'
df['Scheduled Arrival']= pd.to_datetime(df['Scheduled Arrival'], format=FMT)
df['Actual Arrival']= pd.to_datetime(df['Actual Arrival'], format=FMT)
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date,Actual Arrival,Scheduled Arrival
0,EV,2017-10-27,5363.0,N738EV,CVG,01:21,00:00,2017-10-28,2017-10-28,2017-10-27 01:21:00


In [22]:
df['Difference']= (df['Scheduled Arrival']-df['Actual Arrival']).astype('timedelta64[m]')
df

,Carrier Code,Scheduled Date,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Actual Arrival Date,Actual Arrival,Scheduled Arrival,Difference
0,EV,2017-10-27,5363.0,N738EV,CVG,01:21,00:00,2017-10-28,2017-10-28,2017-10-27 01:21:00,-1359.0


In [23]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2019
Count_2019=df[df['Actual Arrival Date'].str.contains("2019")]
df19= pd.DataFrame(Count_2019)
Total_19=len(df19)
Twenty_Mins_Early_19= len(df19[df19['Difference'] > 20.0])
Early_Arrivals_19= len(df19[df19['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2019:",Total_19)
print("Flights that arrived early in 2019:",Early_Arrivals_19)
print("Flights that arrived more than 20 mins early in 2019:",Twenty_Mins_Early_19)

Total Alaska Airlines flights that landed at JFK in 2019: 0
Flights that arrived early in 2019: 0
Flights that arrived more than 20 mins early in 2019: 0


In [24]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2018
Count_2018=df[df['Actual Arrival Date'].str.contains("2018")]
df18= pd.DataFrame(Count_2018)
Total_18=len(df18)
Twenty_Mins_Early_18= len(df18[df18['Difference'] > 20.0])
Early_Arrivals_18= len(df18[df18['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2018:",Total_18)
print("Flights that arrived early in 2018:",Early_Arrivals_18)
print("Flights that arrived more than 20 mins early in 2018:",Twenty_Mins_Early_18)

Total Alaska Airlines flights that landed at JFK in 2018: 0
Flights that arrived early in 2018: 0
Flights that arrived more than 20 mins early in 2018: 0


In [25]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2017
Count_2017=df[df['Actual Arrival Date'].str.contains("2017")]
df17= pd.DataFrame(Count_2017)
Total_17=len(df17)
Twenty_Mins_Early_17= len(df17[df17['Difference'] > 20.0])
Early_Arrivals_17= len(df17[df17['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2017:",Total_17)
print("Flights that arrived early in 2017:",Early_Arrivals_17)
print("Flights that arrived more than 20 mins early in 2017:",Twenty_Mins_Early_17)

Total Alaska Airlines flights that landed at JFK in 2017: 1
Flights that arrived early in 2017: 0
Flights that arrived more than 20 mins early in 2017: 0


In [26]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2016
Count_2016=df[df['Actual Arrival Date'].str.contains("2016")]
df16= pd.DataFrame(Count_2016)
Total_16=len(df16)
Twenty_Mins_Early_16= len(df16[df16['Difference'] > 20.0])
Early_Arrivals_16= len(df16[df16['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2016:",Total_16)
print("Flights that arrived early in 2016:",Early_Arrivals_16)
print("Flights that arrived more than 20 mins early in 2016:",Twenty_Mins_Early_16)

Total Alaska Airlines flights that landed at JFK in 2016: 0
Flights that arrived early in 2016: 0
Flights that arrived more than 20 mins early in 2016: 0


In [27]:
#Count number of flights with Scheduled Arrival Time higher than Actual Arrival or flights that arrived early in 2015
Count_2015=df[df['Actual Arrival Date'].str.contains("2015")]
df15= pd.DataFrame(Count_2015)
Total_15=len(df15)
Twenty_Mins_Early_15= len(df15[df15['Difference'] > 20.0])
Early_Arrivals_15= len(df15[df15['Difference'] > 0])
print("Total Alaska Airlines flights that landed at JFK in 2015:",Total_15)
print("Flights that arrived early in 2015:",Early_Arrivals_15)
print("Flights that arrived more than 20 mins early in 2015:",Twenty_Mins_Early_15)

Total Alaska Airlines flights that landed at JFK in 2015: 0
Flights that arrived early in 2015: 0
Flights that arrived more than 20 mins early in 2015: 0


In [28]:


# dictionary of lists  
 
year =["2015","2016","2017","2018","2019"]
total_flights= [Total_15, Total_16, Total_17, Total_18, Total_19]
early_arrival=[Early_Arrivals_15,Early_Arrivals_16,Early_Arrivals_17,Early_Arrivals_18,Early_Arrivals_19]
inflated_padding=[Twenty_Mins_Early_15,Twenty_Mins_Early_16,Twenty_Mins_Early_17,Twenty_Mins_Early_18,Twenty_Mins_Early_19]

dict = {'year': year, 'Total flights': total_flights, 'Early Arrivals': early_arrival, 'Inflated padding(>20mins early)': inflated_padding} 
df = pd.DataFrame(dict) 
  
# saving the dataframe 
df.to_csv(r'../output/Atlantic_Southeast.csv',index=False) 